In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os
from PIL import Image
import keras
from keras.models import Model
from keras.layers import Conv2D, MaxPooling2D, Input, Conv2DTranspose, Concatenate, BatchNormalization, UpSampling2D
from keras.layers import  Dropout, Activation
from tensorflow.keras.optimizers import Adam, SGD
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
from keras import backend as K
from tensorflow.keras.utils import plot_model
import tensorflow as tf
import glob
import random
import cv2
from random import shuffle
from osgeo import gdal

from pyproj import Proj, transform
from tqdm import tqdm
from shapely.geometry import Polygon

import os
import numpy as np
import tensorflow as tf
from glob import glob
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, concatenate, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, CSVLogger, TensorBoard

In [ ]:
import os
import numpy as np
import cv2
from glob import glob
import tensorflow as tf
from osgeo import gdal
from sklearn.model_selection import train_test_split

path = "D:/Kanwil Riau/Updating NPGT Perkebunan/OLAH TIM/RIAU"
images = sorted(glob(os.path.join(path, "Image/*")))
masks = sorted(glob(os.path.join(path, "Mask/*")))

minx = 0
maxx = 65535

a = images[1]
b = gdal.Open(a).ReadAsArray()
c = np.rollaxis(b, 0, 3)
d = (c - minx) / (maxx - minx)
#d = np.expand_dims(c, axis=-3)
print(d.shape)

e = masks[1]
f = gdal.Open(e).ReadAsArray()
g = np.expand_dims(f, axis=-1)
g.shape

#h = (g - minx) / (maxx - minx)


In [ ]:
from sklearn.model_selection import train_test_split
def load_data(path, split=0.1):
    images = sorted(glob(os.path.join(path, "Image/*")))
    masks = sorted(glob(os.path.join(path, "Mask/*")))

    total_size = len(images)
    valid_size = int(split * total_size)
    test_size = int(split * total_size)

    train_x, valid_x = train_test_split(images, test_size=valid_size)
    train_y, valid_y = train_test_split(masks, test_size=valid_size)

    train_x, test_x = train_test_split(train_x, test_size=test_size)
    train_y, test_y = train_test_split(train_y, test_size=test_size)

    return (train_x, train_y), (valid_x, valid_y), (test_x, test_y)


In [ ]:
def read_image(path):
    #path = path.decode()
    x = gdal.Open(path).ReadAsArray()
    x = np.rollaxis(x, 0, 3)
    x = x.astype(np.float32)
    return 


In [ ]:
def read_mask(path):
    path = path.decode()
    y = gdal.Open(path).ReadAsArray()
    #x[x == 2] = 1 
    #x = np.rollaxis(x, 0, 3)
    #x[np.isnan(x)] = 0
    y = np.expand_dims(y, axis=-1)
    y = y.astype(np.float32)
    return y

In [ ]:
def preprocess_mask(mask, num_classes=13):
    mask = tf.squeeze(mask, axis=-1)  # Menghapus dimensi channel jika ada
    mask = tf.cast(mask, tf.int32)    # Pastikan mask dalam format integer
    mask = tf.one_hot(mask, num_classes)  # One-hot encoding
    mask = tf.cast(mask, tf.float32)  # Konversi ke float32
    return mask


In [ ]:
def tf_parse(x, y, num_classes=13):
    def _parse(x, y):
        x = read_image(x)
        y = read_mask(y)
        y = preprocess_mask(y, num_classes)  # Ensure masks are one-hot encoded
        return x, y

    x, y = tf.numpy_function(_parse, [x, y], [tf.float32, tf.float32])
    x.set_shape([1024, 1024, 4])
    y.set_shape([1024, 1024, num_classes])
    
    return x, y

In [ ]:
def tf_dataset(x, y, batch=8, buffer_size=100, num_classes=13):
    dataset = tf.data.Dataset.from_tensor_slices((x, y))
    dataset = dataset.map(lambda x, y: tf_parse(x, y, num_classes), num_parallel_calls=tf.data.AUTOTUNE)
    dataset = dataset.shuffle(buffer_size)
    dataset = dataset.batch(batch)
    dataset = dataset.prefetch(tf.data.AUTOTUNE)
    dataset = dataset.repeat()
    return dataset

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, MaxPooling2D, UpSampling2D, Concatenate, Input, Dropout, BatchNormalization, Activation
from tensorflow.keras.models import Model

def unet(input_shape=(1024, 1024, 4), num_classes=13):
    inputs = Input(shape=input_shape)

    # Encoder
    conv1 = Conv2D(64, (3, 3), padding='same')(inputs)
    conv1 = BatchNormalization()(conv1)
    conv1 = Activation('relu')(conv1)
    conv1 = Conv2D(64, (3, 3), padding='same')(conv1)
    conv1 = BatchNormalization()(conv1)
    conv1 = Activation('relu')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

    conv2 = Conv2D(128, (3, 3), padding='same')(pool1)
    conv2 = BatchNormalization()(conv2)
    conv2 = Activation('relu')(conv2)
    conv2 = Conv2D(128, (3, 3), padding='same')(conv2)
    conv2 = BatchNormalization()(conv2)
    conv2 = Activation('relu')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

    conv3 = Conv2D(256, (3, 3), padding='same')(pool2)
    conv3 = BatchNormalization()(conv3)
    conv3 = Activation('relu')(conv3)
    conv3 = Conv2D(256, (3, 3), padding='same')(conv3)
    conv3 = BatchNormalization()(conv3)
    conv3 = Activation('relu')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)

    conv4 = Conv2D(512, (3, 3), padding='same')(pool3)
    conv4 = BatchNormalization()(conv4)
    conv4 = Activation('relu')(conv4)
    conv4 = Conv2D(512, (3, 3), padding='same')(conv4)
    conv4 = BatchNormalization()(conv4)
    conv4 = Activation('relu')(conv4)
    drop4 = Dropout(0.3)(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(drop4)

    conv5 = Conv2D(1024, (3, 3), padding='same')(pool4)
    conv5 = BatchNormalization()(conv5)
    conv5 = Activation('relu')(conv5)
    conv5 = Conv2D(1024, (3, 3), padding='same')(conv5)
    conv5 = BatchNormalization()(conv5)
    conv5 = Activation('relu')(conv5)
    drop5 = Dropout(0.3)(conv5)

    # Decoder
    up6 = UpSampling2D(size=(2, 2))(drop5)
    up6 = Conv2D(512, (2, 2), padding='same')(up6)
    merge6 = Concatenate()([drop4, up6])
    conv6 = Conv2D(512, (3, 3), padding='same')(merge6)
    conv6 = BatchNormalization()(conv6)
    conv6 = Activation('relu')(conv6)
    conv6 = Conv2D(512, (3, 3), padding='same')(conv6)
    conv6 = BatchNormalization()(conv6)
    conv6 = Activation('relu')(conv6)

    up7 = UpSampling2D(size=(2, 2))(conv6)
    up7 = Conv2D(256, (2, 2), padding='same')(up7)
    merge7 = Concatenate()([conv3, up7])
    conv7 = Conv2D(256, (3, 3), padding='same')(merge7)
    conv7 = BatchNormalization()(conv7)
    conv7 = Activation('relu')(conv7)
    conv7 = Conv2D(256, (3, 3), padding='same')(conv7)
    conv7 = BatchNormalization()(conv7)
    conv7 = Activation('relu')(conv7)

    up8 = UpSampling2D(size=(2, 2))(conv7)
    up8 = Conv2D(128, (2, 2), padding='same')(up8)
    merge8 = Concatenate()([conv2, up8])
    conv8 = Conv2D(128, (3, 3), padding='same')(merge8)
    conv8 = BatchNormalization()(conv8)
    conv8 = Activation('relu')(conv8)
    conv8 = Conv2D(128, (3, 3), padding='same')(conv8)
    conv8 = BatchNormalization()(conv8)
    conv8 = Activation('relu')(conv8)

    up9 = UpSampling2D(size=(2, 2))(conv8)
    up9 = Conv2D(64, (2, 2), padding='same')(up9)
    merge9 = Concatenate()([conv1, up9])
    conv9 = Conv2D(64, (3, 3), padding='same')(merge9)
    conv9 = BatchNormalization()(conv9)
    conv9 = Activation('relu')(conv9)
    conv9 = Conv2D(64, (3, 3), padding='same')(conv9)
    conv9 = BatchNormalization()(conv9)
    conv9 = Activation('relu')(conv9)

    # Output Layer
    output = Conv2D(num_classes, (1, 1), activation='softmax')(conv9)

    model = Model(inputs=inputs, outputs=output)

    return model

In [ ]:
model = unet()
model.summary()

In [ ]:
import os
import numpy as np
import cv2
from glob import glob
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, CSVLogger, TensorBoard

In [ ]:
import tensorflow as tf

def iou_multiclass(num_classes=13):
    def iou(y_true, y_pred):
        # Mengambil prediksi dengan nilai tertinggi untuk setiap kelas
        y_pred = tf.argmax(y_pred, axis=-1)
        y_true = tf.argmax(y_true, axis=-1)

        iou_per_class = []
        for i in range(num_classes):
            # Menghitung intersection
            intersection = tf.reduce_sum(tf.cast((y_pred == i) & (y_true == i), tf.float32))
            # Menghitung union
            union = tf.reduce_sum(tf.cast((y_pred == i) | (y_true == i), tf.float32))
            
            # Menghindari pembagian dengan nol
            iou_class = tf.cond(tf.equal(union, 0), lambda: tf.constant(1.0), lambda: intersection / union)
            iou_per_class.append(iou_class)
        
        # Menghitung rata-rata IoU untuk semua kelas
        mean_iou = tf.reduce_mean(iou_per_class)
        return mean_iou

    return iou




In [ ]:
if __name__ == "__main__":
    # Dataset
    path = "D:/Kanwil Riau/Updating NPGT Perkebunan/OLAH TIM/RIAU"
    (train_x, train_y), (valid_x, valid_y), (test_x, test_y) = load_data(path)

In [ ]:
batch = 8
epochs = 100

In [ ]:
    # Create TensorFlow datasets
train_dataset = tf_dataset(train_x, train_y)
valid_dataset = tf_dataset(valid_x, valid_y)
test_dataset = tf_dataset(test_x, test_y, batch=1)  # Test dataset usually has batch size 1
print(train_dataset)
print(valid_dataset)
print(test_dataset)

In [ ]:
model = unet()
opt = tf.keras.optimizers.Adam()
metrics = ["accuracy", tf.keras.metrics.Recall(), tf.keras.metrics.Precision(),iou_multiclass(num_classes=13)]
model.compile(loss="categorical_crossentropy", optimizer=opt, metrics=metrics)

In [ ]:
callbacks = [
    ModelCheckpoint("Files/RIAU.keras"),
    ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=4),
    CSVLogger("Files/RIAU.csv"),
    TensorBoard(),
    EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
]

In [ ]:
train_steps = len(train_x)//batch
valid_steps = len(valid_x)//batch

if len(train_x) % batch != 0:
    train_steps += 1
if len(valid_x) % batch != 0:
    valid_steps += 1

In [ ]:
model.fit(train_dataset,
          validation_data=valid_dataset,
          epochs=100,
          steps_per_epoch=train_steps,
          validation_steps=valid_steps,
          callbacks=callbacks)

In [ ]:
import os
import numpy as np
import cv2
import tensorflow as tf
from tensorflow.keras.utils import CustomObjectScope
from tqdm import tqdm

In [ ]:
def mask_parse(mask):
    mask = np.squeeze(mask)
    mask = [mask, mask, mask]
    mask = np.transpose(mask, (1, 2, 0))
    return mask

In [ ]:
model.save("NPGT_Riau.keras")

In [ ]:
batch_size = 8
test_dataset = tf_dataset(test_x, test_y, batch=batch_size)
 
test_steps = (len(test_x)//batch_size)
if len(test_x) % batch_size != 0:
    test_steps += 1

In [ ]:
model.evaluate(test_dataset, steps=test_steps)